# Episode 9: Errors and Exceptions

Errors are inevitable in programming, but Python provides powerful tools to handle them gracefully. In this notebook, we'll learn to read error messages, handle exceptions, and write robust code for inflammation data analysis.

## Learning Objectives
- Understand different types of errors in Python
- Read and interpret error messages and tracebacks
- Use try/except blocks to handle exceptions
- Implement error handling in data analysis workflows
- Write defensive code that anticipates problems
- Create custom exceptions for specific use cases

## Introduction

When analyzing inflammation data, many things can go wrong: files might be missing, data might be corrupted, or calculations might encounter unexpected values. Learning to handle these situations gracefully is crucial for robust data analysis.

## 1. Types of Errors

Python has several types of errors - let's see them in action:

In [ ]:
# Syntax Errors - caught before the program runs
print("Syntax errors prevent code from running at all")
print("Examples (these are commented out because they would break the notebook):")
print("# print('missing closing quote)")
print("# if True  # missing colon")
print("# def function(  # missing closing parenthesis")

# These will work:
print("This line executes correctly")
if True:
    print("Proper syntax with colon and indentation")

In [ ]:
# Runtime Errors - occur during execution
print("Runtime errors happen while the program is running:")

# NameError - using undefined variables
try:
    print(undefined_variable)
except NameError as e:
    print(f"NameError: {e}")

# TypeError - wrong data type for operation
try:
    result = "text" + 42
except TypeError as e:
    print(f"TypeError: {e}")

# IndexError - accessing invalid list index
try:
    inflammation_data = [1.5, 2.3, 1.8]
    invalid_reading = inflammation_data[10]  # Index doesn't exist
except IndexError as e:
    print(f"IndexError: {e}")

# KeyError - accessing invalid dictionary key
try:
    patient_info = {'id': 'P001', 'age': 25}
    weight = patient_info['weight']  # Key doesn't exist
except KeyError as e:
    print(f"KeyError: {e}")

In [ ]:
# Mathematical errors
try:
    result = 10 / 0  # Division by zero
except ZeroDivisionError as e:
    print(f"ZeroDivisionError: {e}")

try:
    import math
    result = math.sqrt(-1)  # Invalid mathematical operation
except ValueError as e:
    print(f"ValueError: {e}")

# File-related errors
try:
    with open('nonexistent_file.csv', 'r') as file:
        data = file.read()
except FileNotFoundError as e:
    print(f"FileNotFoundError: {e}")

## 2. Reading Error Messages and Tracebacks

Understanding error messages is crucial for debugging:

In [ ]:
# Function that will cause an error
def calculate_average_inflammation(patient_data):
    """Calculate average inflammation for a patient."""
    total = sum(patient_data)
    count = len(patient_data)
    return total / count

def analyze_patient(patient_id, data):
    """Analyze a single patient's data."""
    print(f"Analyzing patient {patient_id}")
    average = calculate_average_inflammation(data)
    return {
        'patient_id': patient_id,
        'average_inflammation': average,
        'status': 'analyzed'
    }

def process_study_data(study_data):
    """Process data for multiple patients."""
    results = []
    for patient_id, data in study_data.items():
        result = analyze_patient(patient_id, data)
        results.append(result)
    return results

# Test with good data first
good_data = {
    'P001': [1.5, 2.3, 1.8, 2.1],
    'P002': [2.8, 3.5, 4.1, 3.2]
}

print("Processing good data:")
results = process_study_data(good_data)
for result in results:
    print(f"  {result['patient_id']}: avg = {result['average_inflammation']:.2f}")

In [ ]:
# Now let's see what happens with problematic data
problematic_data = {
    'P003': [1.5, 2.3, 1.8],
    'P004': [],  # Empty data - will cause ZeroDivisionError
    'P005': [3.2, 4.1, 2.8]
}

print("Processing problematic data (this will cause an error):")
try:
    results = process_study_data(problematic_data)
except ZeroDivisionError as e:
    print(f"\n🚨 Error occurred: {e}")
    print("\n📍 Error Analysis:")
    print("   - The error occurred in calculate_average_inflammation()")
    print("   - It was called from analyze_patient()")
    print("   - Which was called from process_study_data()")
    print("   - The problem: Patient P004 has empty data (division by zero)")
    
    import traceback
    print("\n📋 Full traceback:")
    traceback.print_exc()

## 3. Basic Exception Handling

Using try/except blocks to handle errors gracefully:

In [ ]:
# Basic try/except structure
def safe_divide(a, b):
    """Safely divide two numbers."""
    try:
        result = a / b
        return result
    except ZeroDivisionError:
        print(f"Warning: Cannot divide {a} by zero")
        return None

# Test the safe division
print("Safe division examples:")
print(f"10 / 2 = {safe_divide(10, 2)}")
print(f"10 / 0 = {safe_divide(10, 0)}")
print(f"7 / 3 = {safe_divide(7, 3)}")

In [ ]:
# Improved inflammation analysis with error handling
def safe_calculate_average(data, patient_id="Unknown"):
    """Safely calculate average with error handling."""
    try:
        if not data:
            raise ValueError("No data provided")
        
        # Check for non-numeric data
        numeric_data = []
        for item in data:
            if not isinstance(item, (int, float)):
                raise TypeError(f"Non-numeric data found: {item}")
            numeric_data.append(item)
        
        average = sum(numeric_data) / len(numeric_data)
        return average
        
    except ValueError as e:
        print(f"⚠️  Data error for patient {patient_id}: {e}")
        return None
    except TypeError as e:
        print(f"⚠️  Type error for patient {patient_id}: {e}")
        return None
    except Exception as e:
        print(f"⚠️  Unexpected error for patient {patient_id}: {e}")
        return None

# Test with various problematic data
test_cases = [
    ("P001", [1.5, 2.3, 1.8]),           # Good data
    ("P002", []),                         # Empty data
    ("P003", [1.5, "invalid", 2.3]),      # Non-numeric data
    ("P004", [2.1, 3.4, 2.8, 1.9]),      # Good data
    ("P005", None),                       # None instead of list
]

print("Testing safe average calculation:")
for patient_id, data in test_cases:
    try:
        avg = safe_calculate_average(data, patient_id)
        if avg is not None:
            print(f"✅ {patient_id}: Average = {avg:.2f}")
        else:
            print(f"❌ {patient_id}: Could not calculate average")
    except Exception as e:
        print(f"💥 {patient_id}: Unhandled error = {e}")

## 4. Multiple Exception Types

Handling different types of errors with specific responses:

In [ ]:
def load_and_process_patient_data(patient_id, data_source):
    """Load and process patient data with comprehensive error handling."""
    
    try:
        # Simulate different data sources
        if data_source == "file":
            # Simulate file reading
            filename = f"patient_{patient_id}.csv"
            with open(filename, 'r') as f:
                data = [float(line.strip()) for line in f]
                
        elif data_source == "database":
            # Simulate database connection
            if patient_id == "P999":
                raise ConnectionError("Database connection failed")
            data = [1.5, 2.3, 1.8, 2.1]  # Mock data
            
        elif data_source == "api":
            # Simulate API call
            if patient_id == "P404":
                raise ValueError("Patient not found in API")
            data = [2.8, 3.5, 4.1, 3.2]  # Mock data
            
        else:
            raise ValueError(f"Unknown data source: {data_source}")
        
        # Process the data
        if not data:
            raise ValueError("No data found for patient")
        
        average = sum(data) / len(data)
        maximum = max(data)
        minimum = min(data)
        
        return {
            'patient_id': patient_id,
            'data_source': data_source,
            'readings': data,
            'statistics': {
                'average': average,
                'minimum': minimum,
                'maximum': maximum,
                'count': len(data)
            },
            'status': 'success'
        }
        
    except FileNotFoundError:
        return {
            'patient_id': patient_id,
            'status': 'file_not_found',
            'error': f"Patient file not found: patient_{patient_id}.csv",
            'suggestion': 'Check if file exists or use different data source'
        }
        
    except ConnectionError as e:
        return {
            'patient_id': patient_id,
            'status': 'connection_error',
            'error': str(e),
            'suggestion': 'Try again later or use backup data source'
        }
        
    except ValueError as e:
        return {
            'patient_id': patient_id,
            'status': 'data_error',
            'error': str(e),
            'suggestion': 'Verify patient ID and data source'
        }
        
    except Exception as e:
        return {
            'patient_id': patient_id,
            'status': 'unknown_error',
            'error': f"Unexpected error: {type(e).__name__}: {e}",
            'suggestion': 'Contact system administrator'
        }

# Test different error scenarios
test_scenarios = [
    ("P001", "database"),     # Success
    ("P002", "api"),          # Success
    ("P999", "database"),     # Connection error
    ("P404", "api"),          # Patient not found
    ("P003", "file"),         # File not found
    ("P004", "invalid"),      # Invalid data source
]

print("Testing different error scenarios:")
print("=" * 50)

for patient_id, data_source in test_scenarios:
    result = load_and_process_patient_data(patient_id, data_source)
    
    if result['status'] == 'success':
        stats = result['statistics']
        print(f"✅ {patient_id} ({data_source}): avg={stats['average']:.2f}, "
              f"range={stats['minimum']:.1f}-{stats['maximum']:.1f}")
    else:
        print(f"❌ {patient_id} ({data_source}): {result['status']}")
        print(f"   Error: {result['error']}")
        print(f"   Suggestion: {result['suggestion']}")
    print()

## 5. Finally and Else Blocks

Complete exception handling with cleanup and success actions:

In [ ]:
def process_inflammation_file(filename, backup_data=None):
    """Process inflammation data with complete exception handling."""
    
    file_handle = None
    processing_start_time = None
    
    try:
        print(f"📁 Attempting to open {filename}...")
        
        # Simulate file opening (we'll use mock data)
        if filename == "missing_file.csv":
            raise FileNotFoundError(f"File {filename} not found")
        elif filename == "corrupted_file.csv":
            raise ValueError("File contains corrupted data")
        elif filename == "permission_denied.csv":
            raise PermissionError("Permission denied to read file")
        
        # Mock successful file reading
        import time
        processing_start_time = time.time()
        
        print(f"✅ File opened successfully")
        
        # Simulate data processing
        mock_data = [1.5, 2.3, 1.8, 2.1, 1.9, 2.5, 1.7]
        
        # Simulate some processing time
        time.sleep(0.1)
        
        # Process data
        total = sum(mock_data)
        count = len(mock_data)
        average = total / count
        
        result = {
            'filename': filename,
            'data': mock_data,
            'average': average,
            'total': total,
            'count': count,
            'source': 'file'
        }
        
        return result
        
    except FileNotFoundError as e:
        print(f"❌ File error: {e}")
        if backup_data:
            print(f"🔄 Using backup data instead")
            return {
                'filename': 'backup_data',
                'data': backup_data,
                'average': sum(backup_data) / len(backup_data),
                'total': sum(backup_data),
                'count': len(backup_data),
                'source': 'backup'
            }
        return None
        
    except (ValueError, PermissionError) as e:
        print(f"❌ Processing error: {e}")
        return None
        
    except Exception as e:
        print(f"❌ Unexpected error: {type(e).__name__}: {e}")
        return None
        
    else:
        # This runs only if no exception occurred
        print(f"🎉 File processed successfully!")
        
    finally:
        # This always runs, regardless of exceptions
        if file_handle:
            print(f"🔒 Closing file handle")
            # file_handle.close()  # In real code
        
        if processing_start_time:
            processing_time = time.time() - processing_start_time
            print(f"⏱️  Processing took {processing_time:.3f} seconds")
        
        print(f"🧹 Cleanup completed for {filename}")
        print("-" * 40)

# Test the complete exception handling
backup_data = [2.0, 2.5, 2.2, 2.8, 2.1]

test_files = [
    "good_file.csv",
    "missing_file.csv",
    "corrupted_file.csv",
    "permission_denied.csv"
]

print("Testing complete exception handling:")
print("=" * 50)

for filename in test_files:
    result = process_inflammation_file(filename, backup_data)
    if result:
        print(f"📊 Result: avg={result['average']:.2f} from {result['source']}")
    else:
        print(f"💥 Failed to process {filename}")
    print()

### Exercise 9.1
Create a robust data validation function that:
1. Handles multiple types of input errors
2. Provides specific error messages
3. Returns detailed validation results
4. Suggests fixes for common problems

In [ ]:
# Exercise 9.1 - Your robust validation function
def validate_inflammation_data(data, patient_id="Unknown", strict=True):
    """
    Comprehensive validation of inflammation data.
    
    Parameters:
    - data: the data to validate
    - patient_id: identifier for error reporting
    - strict: whether to enforce strict validation rules
    
    Returns:
    Dictionary with validation results, errors, warnings, and suggestions
    """
    # Your implementation here
    pass

# Test your validation function
test_data_sets = [
    ([1.5, 2.3, 1.8, 2.1], "P001"),           # Good data
    ([], "P002"),                              # Empty
    ([1.5, -2.3, 1.8], "P003"),               # Negative values
    ([1.5, "invalid", 1.8], "P004"),          # Non-numeric
    ([1.5, 2.3, 15.8, 2.1], "P005"),         # Outlier
    (None, "P006"),                           # None
    ("not a list", "P007"),                   # Wrong type
]

# Add your test code here

## 6. Custom Exceptions

Creating specific exception types for your domain:

In [ ]:
# Define custom exceptions for inflammation analysis
class InflammationDataError(Exception):
    """Base exception for inflammation data problems."""
    pass

class EmptyDataError(InflammationDataError):
    """Raised when no inflammation data is provided."""
    def __init__(self, patient_id):
        self.patient_id = patient_id
        super().__init__(f"No inflammation data found for patient {patient_id}")

class InvalidReadingError(InflammationDataError):
    """Raised when inflammation readings are invalid."""
    def __init__(self, patient_id, reading, position):
        self.patient_id = patient_id
        self.reading = reading
        self.position = position
        super().__init__(
            f"Invalid reading '{reading}' at position {position} for patient {patient_id}"
        )

class CriticalInflammationError(InflammationDataError):
    """Raised when inflammation levels are critically high."""
    def __init__(self, patient_id, level, threshold=10.0):
        self.patient_id = patient_id
        self.level = level
        self.threshold = threshold
        super().__init__(
            f"Critical inflammation level {level:.2f} (>{threshold}) detected for patient {patient_id}"
        )

class DataQualityError(InflammationDataError):
    """Raised when data quality is insufficient for analysis."""
    def __init__(self, patient_id, issues):
        self.patient_id = patient_id
        self.issues = issues
        issue_text = ", ".join(issues)
        super().__init__(
            f"Data quality issues for patient {patient_id}: {issue_text}"
        )

# Function using custom exceptions
def analyze_inflammation_data(patient_id, readings, critical_threshold=8.0, 
                            min_readings=3, max_gap_ratio=0.3):
    """Analyze inflammation data with custom exception handling."""
    
    # Check for empty data
    if not readings:
        raise EmptyDataError(patient_id)
    
    # Validate and clean readings
    clean_readings = []
    quality_issues = []
    
    for i, reading in enumerate(readings):
        # Check for non-numeric data
        if not isinstance(reading, (int, float)):
            raise InvalidReadingError(patient_id, reading, i)
        
        # Check for negative values
        if reading < 0:
            raise InvalidReadingError(patient_id, reading, i)
        
        # Check for critical levels
        if reading > critical_threshold:
            raise CriticalInflammationError(patient_id, reading, critical_threshold)
        
        # Note quality issues
        if reading == 0:
            quality_issues.append(f"zero reading at position {i}")
        elif reading > 6.0:
            quality_issues.append(f"high reading {reading} at position {i}")
        
        clean_readings.append(reading)
    
    # Check minimum data requirements
    if len(clean_readings) < min_readings:
        quality_issues.append(f"insufficient readings ({len(clean_readings)} < {min_readings})")
    
    # Check for data gaps (too many zeros)
    zero_count = clean_readings.count(0)
    gap_ratio = zero_count / len(clean_readings)
    if gap_ratio > max_gap_ratio:
        quality_issues.append(f"too many gaps ({gap_ratio:.1%} zeros)")
    
    # Raise quality error if issues found
    if quality_issues:
        raise DataQualityError(patient_id, quality_issues)
    
    # Calculate statistics
    average = sum(clean_readings) / len(clean_readings)
    maximum = max(clean_readings)
    minimum = min(clean_readings)
    
    return {
        'patient_id': patient_id,
        'readings': clean_readings,
        'statistics': {
            'average': average,
            'minimum': minimum,
            'maximum': maximum,
            'count': len(clean_readings)
        },
        'status': 'success'
    }

# Test custom exceptions
test_cases = [
    ("P001", [1.5, 2.3, 1.8, 2.1]),                    # Good data
    ("P002", []),                                        # Empty data
    ("P003", [1.5, "invalid", 1.8]),                   # Invalid reading
    ("P004", [1.5, -2.3, 1.8]),                        # Negative reading
    ("P005", [1.5, 2.3, 9.8, 2.1]),                    # Critical level
    ("P006", [1.5, 2.3]),                               # Too few readings
    ("P007", [0, 0, 1.5, 0, 0]),                       # Too many gaps
    ("P008", [1.5, 2.3, 6.8, 2.1, 1.9]),              # High but not critical
]

print("Testing custom exceptions:")
print("=" * 50)

for patient_id, readings in test_cases:
    try:
        result = analyze_inflammation_data(patient_id, readings)
        stats = result['statistics']
        print(f"✅ {patient_id}: avg={stats['average']:.2f}, "
              f"range={stats['minimum']:.1f}-{stats['maximum']:.1f}")
        
    except EmptyDataError as e:
        print(f"📭 {e}")
        
    except InvalidReadingError as e:
        print(f"🚫 {e}")
        print(f"    Suggestion: Check data at position {e.position}")
        
    except CriticalInflammationError as e:
        print(f"🚨 {e}")
        print(f"    Action required: Immediate medical attention!")
        
    except DataQualityError as e:
        print(f"⚠️  {e}")
        print(f"    Issues found: {len(e.issues)}")
        
    except InflammationDataError as e:
        print(f"❌ General inflammation data error: {e}")
        
    except Exception as e:
        print(f"💥 Unexpected error for {patient_id}: {type(e).__name__}: {e}")
    
    print()

## 7. Error Handling in Data Processing Pipelines

Building robust data analysis workflows:

In [ ]:
# Comprehensive data processing pipeline with error handling
class InflammationAnalyzer:
    """A robust inflammation data analyzer with comprehensive error handling."""
    
    def __init__(self, strict_mode=True, auto_fix=True):
        self.strict_mode = strict_mode
        self.auto_fix = auto_fix
        self.processing_log = []
        self.error_count = 0
        self.warning_count = 0
        
    def log_message(self, level, message, patient_id=None):
        """Log processing messages."""
        import datetime
        timestamp = datetime.datetime.now().strftime("%H:%M:%S")
        
        if patient_id:
            log_entry = f"[{timestamp}] {level}: {patient_id} - {message}"
        else:
            log_entry = f"[{timestamp}] {level}: {message}"
            
        self.processing_log.append(log_entry)
        
        if level == "ERROR":
            self.error_count += 1
        elif level == "WARNING":
            self.warning_count += 1
    
    def clean_reading(self, reading, position, patient_id):
        """Clean individual readings with error handling."""
        try:
            # Convert to float if possible
            if isinstance(reading, str):
                # Try to clean common string issues
                reading = reading.strip().replace(',', '.')
                if reading.lower() in ['na', 'null', 'none', '']:
                    if self.auto_fix:
                        self.log_message("WARNING", f"Missing value at position {position}, using 0.0", patient_id)
                        return 0.0
                    else:
                        raise InvalidReadingError(patient_id, reading, position)
                
            numeric_reading = float(reading)
            
            # Check for negative values
            if numeric_reading < 0:
                if self.auto_fix:
                    self.log_message("WARNING", f"Negative value {numeric_reading} at position {position}, using absolute value", patient_id)
                    return abs(numeric_reading)
                else:
                    raise InvalidReadingError(patient_id, numeric_reading, position)
            
            # Check for extreme values
            if numeric_reading > 15.0:
                if self.auto_fix:
                    self.log_message("WARNING", f"Extreme value {numeric_reading} at position {position}, capping at 15.0", patient_id)
                    return 15.0
                else:
                    self.log_message("WARNING", f"Extreme value {numeric_reading} at position {position}", patient_id)
            
            return numeric_reading
            
        except (ValueError, TypeError) as e:
            if self.auto_fix:
                self.log_message("WARNING", f"Invalid value '{reading}' at position {position}, using 0.0", patient_id)
                return 0.0
            else:
                raise InvalidReadingError(patient_id, reading, position)
    
    def process_patient(self, patient_id, raw_data):
        """Process a single patient with comprehensive error handling."""
        try:
            self.log_message("INFO", f"Starting processing", patient_id)
            
            # Validate input
            if raw_data is None:
                raise EmptyDataError(patient_id)
            
            if not isinstance(raw_data, (list, tuple)):
                raise TypeError(f"Data must be a list or tuple, got {type(raw_data).__name__}")
            
            if len(raw_data) == 0:
                raise EmptyDataError(patient_id)
            
            # Clean each reading
            cleaned_readings = []
            for i, reading in enumerate(raw_data):
                try:
                    cleaned_reading = self.clean_reading(reading, i, patient_id)
                    cleaned_readings.append(cleaned_reading)
                except InflammationDataError:
                    if self.strict_mode:
                        raise
                    else:
                        # Skip invalid readings in non-strict mode
                        self.log_message("WARNING", f"Skipping invalid reading at position {i}", patient_id)
                        continue
            
            # Check if we have enough data left
            if len(cleaned_readings) == 0:
                raise EmptyDataError(patient_id)
            
            if len(cleaned_readings) < 3:
                if self.strict_mode:
                    raise DataQualityError(patient_id, [f"Only {len(cleaned_readings)} valid readings"])
                else:
                    self.log_message("WARNING", f"Only {len(cleaned_readings)} readings available", patient_id)
            
            # Calculate statistics
            statistics = {
                'count': len(cleaned_readings),
                'sum': sum(cleaned_readings),
                'average': sum(cleaned_readings) / len(cleaned_readings),
                'minimum': min(cleaned_readings),
                'maximum': max(cleaned_readings),
            }
            
            # Add standard deviation
            if len(cleaned_readings) > 1:
                mean = statistics['average']
                variance = sum((x - mean)**2 for x in cleaned_readings) / (len(cleaned_readings) - 1)
                statistics['std_dev'] = variance ** 0.5
            else:
                statistics['std_dev'] = 0.0
            
            # Check for critical conditions
            if statistics['maximum'] > 10.0:
                self.log_message("ERROR", f"Critical inflammation level detected: {statistics['maximum']:.2f}", patient_id)
            elif statistics['average'] > 5.0:
                self.log_message("WARNING", f"High average inflammation: {statistics['average']:.2f}", patient_id)
            
            self.log_message("INFO", f"Processing completed successfully", patient_id)
            
            return {
                'patient_id': patient_id,
                'status': 'success',
                'original_count': len(raw_data),
                'cleaned_count': len(cleaned_readings),
                'readings': cleaned_readings,
                'statistics': statistics
            }
            
        except InflammationDataError as e:
            self.log_message("ERROR", str(e), patient_id)
            return {
                'patient_id': patient_id,
                'status': 'error',
                'error_type': type(e).__name__,
                'error_message': str(e)
            }
            
        except Exception as e:
            self.log_message("ERROR", f"Unexpected error: {type(e).__name__}: {e}", patient_id)
            return {
                'patient_id': patient_id,
                'status': 'error',
                'error_type': type(e).__name__,
                'error_message': str(e)
            }
    
    def process_study(self, study_data):
        """Process entire study with error tracking."""
        results = []
        successful_patients = 0
        failed_patients = 0
        
        self.log_message("INFO", f"Starting study processing: {len(study_data)} patients")
        
        for patient_id, data in study_data.items():
            result = self.process_patient(patient_id, data)
            results.append(result)
            
            if result['status'] == 'success':
                successful_patients += 1
            else:
                failed_patients += 1
        
        self.log_message("INFO", f"Study processing completed: {successful_patients} successful, {failed_patients} failed")
        
        return {
            'results': results,
            'summary': {
                'total_patients': len(study_data),
                'successful': successful_patients,
                'failed': failed_patients,
                'success_rate': successful_patients / len(study_data) if study_data else 0,
                'total_errors': self.error_count,
                'total_warnings': self.warning_count
            },
            'log': self.processing_log
        }

# Test the comprehensive analyzer
study_data = {
    'P001': [1.5, 2.3, 1.8, 2.1],              # Good data
    'P002': [1.5, 'invalid', 1.8, 2.1],       # Invalid reading
    'P003': [1.5, -2.3, 1.8],                 # Negative value
    'P004': [],                                 # Empty
    'P005': [1.5, 2.3, 18.8, 2.1],            # Extreme value
    'P006': [1.5, '', 'NA', 2.1],             # Missing values
    'P007': [1.5, 2.3, 11.8, 2.1],            # Critical level
    'P008': None,                               # None data
}

print("Testing comprehensive analyzer (auto-fix mode):")
print("=" * 60)

analyzer = InflammationAnalyzer(strict_mode=False, auto_fix=True)
study_results = analyzer.process_study(study_data)

# Print results
for result in study_results['results']:
    if result['status'] == 'success':
        stats = result['statistics']
        print(f"✅ {result['patient_id']}: avg={stats['average']:.2f} "
              f"({result['cleaned_count']}/{result['original_count']} readings)")
    else:
        print(f"❌ {result['patient_id']}: {result['error_type']} - {result['error_message']}")

# Print summary
summary = study_results['summary']
print(f"\n📊 Study Summary:")
print(f"   Success rate: {summary['success_rate']:.1%} ({summary['successful']}/{summary['total_patients']})")
print(f"   Errors: {summary['total_errors']}, Warnings: {summary['total_warnings']}")

# Print recent log entries
print(f"\n📋 Recent log entries:")
for entry in study_results['log'][-5:]:
    print(f"   {entry}")

### Final Exercise 9.2
Create a complete error handling system for a data analysis workflow:
1. Handle file I/O errors
2. Validate and clean data with multiple strategies
3. Provide detailed error reporting
4. Implement recovery mechanisms
5. Generate comprehensive logs

In [ ]:
# Final Exercise 9.2 - Your complete error handling system
class RobustInflammationPipeline:
    """A complete, robust inflammation data processing pipeline."""
    
    def __init__(self, config=None):
        # Your initialization here
        pass
    
    def load_data_from_file(self, filename):
        """Load data from file with comprehensive error handling."""
        # Your implementation here
        pass
    
    def validate_and_clean(self, raw_data, patient_id):
        """Validate and clean data with multiple strategies."""
        # Your implementation here
        pass
    
    def analyze_with_recovery(self, data, patient_id):
        """Analyze data with multiple recovery strategies."""
        # Your implementation here
        pass
    
    def generate_error_report(self):
        """Generate comprehensive error and processing report."""
        # Your implementation here
        pass
    
    def process_complete_workflow(self, data_sources):
        """Run the complete pipeline with full error handling."""
        # Your implementation here
        pass

# Test your complete system
# Add comprehensive test cases here

## Summary

In this episode, we learned:
- **Error types**: Syntax errors, runtime errors, and logical errors
- **Reading tracebacks**: Understanding error messages and call stacks
- **Exception handling**: Using try/except/else/finally blocks
- **Multiple exceptions**: Handling different error types appropriately
- **Custom exceptions**: Creating domain-specific error types
- **Robust pipelines**: Building comprehensive error handling systems
- **Logging and recovery**: Tracking errors and implementing fallbacks

Error handling is essential for creating reliable, maintainable data analysis code!